# Model.py

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

import pickle
from pickle import dump, load

In [2]:
df = pd.read_csv("retailnew.csv")

In [3]:
df = df[['DaySum','MonthAvg','MonthSum']]

In [4]:
df.head()

,DaySum,MonthAvg,MonthSum
0,1059,4.945205,361
1,402,7.291667,175
2,462,5.866667,176
3,61,4.250000,17
4,4,8.000000,8


In [5]:
X = df.iloc[:,0:2]
y = df.iloc[:,2]

In [6]:
lr = LinearRegression()

In [7]:
lr.fit(X,y)

LinearRegression()

In [8]:
#Save Model

filename = 'model.sav'

dump(lr,open(filename,'wb'))

print("Model Saved")

Model Saved


In [9]:
# Load the model that you just saved

lr = load(open(filename,'rb'))

print("Model Loaded")

Model Loaded


In [10]:
lr

LinearRegression()

In [11]:
# Saving the data columns from training

filename = 'model_columns.sav'

model_columns = list(X.columns)
dump(model_columns, open(filename,'wb'))

print("Models columns dumped!")

Models columns dumped!


# Flask application (api.py)

In [12]:
# Dependencies
from flask import Flask, request, jsonify
from pickle import dump, load
import traceback
import pandas as pd
import numpy as np

# Your API definition
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    if lr:
        try:
            json_ = request.json
            print(json_)
            query = pd.get_dummies(pd.DataFrame(json_))
            query = query.reindex(columns=model_columns, fill_value=0)

            prediction = list(lr.predict(query))

            return jsonify({'prediction': str(prediction)})

        except:

            return jsonify({'trace': traceback.format_exc()})
    else:
        print ('Train the model first')
        return ('No model here to use')

if __name__ == '__main__':
    try:
        port = int(sys.argv[1]) # This is for a command-line input
    except:
        port = 12345 # If you don't provide any port the port will be set to 12345

    lr = open('model.sav','rb') # Load "model.pkl"
    print ('Model loaded')
    model_columns = open('model_columns.sav','rb') # Load "model_columns.pkl"
    print ('Model columns loaded')

    app.run(port=port, debug=True)

Model loaded
Model columns loaded
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Unit Test

In [15]:
def load_data():
    data = pd.DataFrame([[5,5.0]], columns=['DaySum','MonthAvg'])
    return data
    

In [17]:
def aggregate_mean(df, column):
    return df.groupby("class")[column].mean().to_dict()

In [18]:
def test_aggregate_mean_feature_1():   
    data = load_data()
    expected = {0: 3, 1: 4}
    result = aggregate_mean(data, "feature_1")
    assert expected == result